<a href="https://colab.research.google.com/github/SzaboNikolettaLaura/GAAlgoritmusok/blob/main/Szabo_Nikoletta_GA_bev_algoritmusok_colab_2_alap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# BIOINFORMATIKA - deceptív fitneszfüggvény gyakorlati alkalmazása
**Feladat:** deceptív fitneszfüggvény bevezetés és gyakorlati alkalmazás

---

Deceptív függvény:
* olyan fitness függvény, amelyben a lokálisan növekvő sémák (schema-average fitness) a GA-t a globális optimumtól távolodó irányba vezetik, így a GA jellemzően rossz lokális optimumhoz konvergál.


* globális maximum: 0000
* bármely alacsonyabb rendű szkémára teljesül: ha az összes 0-t 1-re cseréljük, akkor a szkéma átlagfitnesze nő.

* Készítünk egy 8 bites függvényt:
* f8​(x1​,…,x8​)=f4​(x1​,…,x4​)+f4​(x5​,…,x8​), ahol:
* az első 4 bit első blokknak, az utolsó 4 bit második blokknak felel meg.

Globális maximum:
* Az első blokk maximuma 0000, a másodiké is 0000,
* ezért az egész 8 bites maximumpont: 00000000.

Miért deceptív? (építőkocka-értelemben)
* Minden 4 bites blokk külön-külön deceptív (ezt az első projekt feladatban kell majd bizonyítani)
* Egy 8 bites sémánál, ha egy blokkban kicseréljük a 0-kat 1-re, akkor a blokk átlag-fitnesze nő → a teljes függvény blokkra eső része is nő.
* Így a GA-t bitenként, blokkonként „félrevezeti”: a lokális mintázat az 1-esek irányába tol, miközben a globális optimum a teljesen 0-s vektor.

---



## LABOR / FELADATOK CÉLJA

*   Genetikus algoritmus alapelveinek megértése
*   Szkémák (schema) és átlagfitnesz fogalmának gyakorlása
*   Deceptív fitness táj viselkedésének megismerése
*   Kódolás és problémareprezentáció szerepének megértése
*   Paraméterérzékenység vizsgálata
*   Lokális vs. globális optimum fogalmának elmélyítése
*   Kísérleti (empirikus) gondolkodás fejlesztése
*   Kódolási és implementációs készségek fejlesztése Pythonban
*   Kritikus gondolkodás az algoritmusválasztásról

## FELADAT: KÓD MEGÉRTÉS, majd gyakorlatok extra feladatokkal
A saját implementációhoz adok egy alap kódót, hogy ha vki elakadna, legyen minimál segítség, de ez legyen csak iránymutatás a saját megoldáshoz.

In [1]:
from itertools import product

# 4 bites Whitley-féle DF2 (globális max: 1111, lokális: 0000)
DF2 = {
    "0000": 28, "0001": 26, "0010": 24, "0011": 18,
    "0100": 22, "0101":  6, "0110": 14, "0111":  0,
    "1000": 20, "1001": 12, "1010": 10, "1011":  2,
    "1100":  8, "1101":  4, "1110":  6, "1111": 30,
}

def bit_not(bits: str) -> str:
    return "".join("1" if b == "0" else "0" for b in bits)

# 4 bites deceptív függvény, ahol a globális maximum 0000
f4 = {b: DF2[bit_not(b)] for b in DF2}

def f8(bits: str) -> int:
    """8 bites deceptív fitness: két 4 bites blokk összege."""
    assert len(bits) == 8
    return f4[bits[:4]] + f4[bits[4:]]

# Ellenőrzés: keressük meg a globális maximumot
BITSTRINGS_8 = ["".join(b) for b in product("01", repeat=8)]
values = {x: f8(x) for x in BITSTRINGS_8}
best_val = max(values.values())
best_points = [x for x, v in values.items() if v == best_val]

print("Globális maximum érték (8 bit):", best_val)
print("Globális maximum helye(i):", best_points)  # elvárt: ['00000000']


Globális maximum érték (8 bit): 60
Globális maximum helye(i): ['00000000']


## MELEGÍTŐ FELADAT (könnyű)



1.   Futtassátok a fenti kódot, győződjetek meg róla, hogy a globális maximum tényleg 00000000.
2.   Írjatok GA-t, és figyeljék meg, hogy tipikusan „1-esek felé” sodródik a populáció, nem pedig a 00000000 irányába.





In [2]:
import random
def create_random_person(dna_length=8):
    dna = ""
    for i in range(dna_length):
        choice = random.choice(['0', '1'])
        dna += choice
    return dna

def create_starting_population(pop_size, dna_length=8):
    population = []
    for i in range(pop_size):
        new_person = create_random_person(dna_length)
        population.append(new_person)
    return population

def pick_winner(population, fitness_scores, k=3):
    contestant_indices = random.sample(range(len(population)), k)
    best_index = -1
    highest_score = -1

    for index in contestant_indices:
        score = fitness_scores[index]
        if score > highest_score:
            highest_score = score
            best_index = index

    return population[best_index]

def crossover(parent1, parent2, crossover_rate=0.7):
    if random.random() < crossover_rate:
        cut_point = random.randint(1, len(parent1) - 1)

        child1 = parent1[:cut_point] + parent2[cut_point:]
        child2 = parent2[:cut_point] + parent1[cut_point:]

        return child1, child2

    return parent1, parent2

def mutate(individual, mutation_rate=0.01):
    dna_chars = list(individual)

    for i in range(len(dna_chars)):
        if random.random() < mutation_rate:
            if dna_chars[i] == '0':
                dna_chars[i] = '1'
            else:
                dna_chars[i] = '0'

    return "".join(dna_chars)

def ga(pop_size, max_generations, mutation_rate, crossover_rate, dna_format=8):

    population = create_starting_population(pop_size, dna_format)
    print("Gen   | Max Fit    | Best Person     | Avg Fit")
    print("-" * 50)

    for generation in range(max_generations):

        fitness_scores = []
        for person in population:
            score = f8(person)

            fitness_scores.append(score)

        max_fit = max(fitness_scores)
        avg_fit = sum(fitness_scores) / len(fitness_scores)

        best_person_index = fitness_scores.index(max_fit)
        best_person = population[best_person_index]

        print(f"{generation}     | {max_fit:.2f}       | {best_person} | {avg_fit:.2f}")

        next_generation = []

        while len(next_generation) < pop_size:
            parent1 = pick_winner(population, fitness_scores, 3)
            parent2 = pick_winner(population, fitness_scores, 3)

            child1, child2 = crossover(parent1, parent2, crossover_rate)

            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            next_generation.append(child1)
            next_generation.append(child2)

        population = next_generation[:pop_size]

    print("\nMax generations reached.")

    return best_person

ga(20, 50, 0.05, 0.8, 8)

Gen   | Max Fit    | Best Person     | Avg Fit
--------------------------------------------------
0     | 46.00       | 10111101 | 28.00
1     | 54.00       | 00001101 | 35.30
2     | 50.00       | 10111111 | 40.10
3     | 56.00       | 11111111 | 46.10
4     | 56.00       | 11111111 | 47.60
5     | 56.00       | 11111111 | 49.00
6     | 56.00       | 11111111 | 48.20
7     | 56.00       | 11111111 | 52.20
8     | 56.00       | 11111111 | 54.30
9     | 56.00       | 11111111 | 54.20
10     | 56.00       | 11111111 | 54.50
11     | 56.00       | 11111111 | 53.20
12     | 56.00       | 11111111 | 53.90
13     | 56.00       | 11111111 | 54.80
14     | 56.00       | 11111111 | 54.10
15     | 56.00       | 11111111 | 55.10
16     | 56.00       | 11111111 | 52.50
17     | 56.00       | 11111111 | 53.40
18     | 56.00       | 11111111 | 55.10
19     | 56.00       | 11111111 | 54.30
20     | 56.00       | 11111111 | 53.80
21     | 56.00       | 11111111 | 53.40
22     | 56.00       | 11111111 

'11111111'

## 1) FELADAT (könnyű) – Táblázat kitöltése 4 biten

*   Írd fel táblázatba az összes 4 bites kromoszómát, a hozzájuk tartozó 𝑓4 fitneszértéket.
*   Rendezd őket csökkenő sorrendbe a fitness szerint.
*   Jelöld, hol van a globális maximum és hol a „csapda” (lokális optimum).



In [3]:
print("f4")
for block in f4:
    fitness = f4[block]
    print(block, fitness)

temp_list = []

for block in f4:
    fitness = f4[block]
    pair = [fitness, block]
    temp_list.append(pair)

temp_list.sort(reverse=True)

print("sorted")
for pair in temp_list:
    score = pair[0]
    block = pair[1]
    print(block, score)

f4
0000 30
0001 6
0010 4
0011 8
0100 2
0101 10
0110 12
0111 20
1000 0
1001 14
1010 6
1011 22
1100 18
1101 24
1110 26
1111 28
sorted
0000 30
1111 28
1110 26
1101 24
1011 22
0111 20
1100 18
1001 14
0110 12
0101 10
0011 8
1010 6
0001 6
0010 4
0100 2
1000 0


In [ ]:
# globalis maximum 0000 - 30
# lokalis maximum 1001 - 14

## 2) FELADAT (könnyű) – Séma-átlag kézzel

*   Válaszd a sémát: 0***, számold ki kézzel a hozzá tartozó átlagfitneszt.
*   Utána nézd meg a 1*** sémát, és hasonlítsd össze.
*   Kövesd végig még 2–3 sémára (pl. 00** → 11**, * 0 * 0 → * 1 * 1 ).


In [ ]:
DF2 = {
    "0000": 28, "0001": 26, "0010": 24, "0011": 18,
    "0100": 22, "0101":  6, "0110": 14, "0111":  0,
    "1000": 20, "1001": 12, "1010": 10, "1011":  2,
    "1100":  8, "1101":  4, "1110":  6, "1111": 30,
}
# 0*** = 0000, 0001, 0010, 0011, 0100, 0101, 0110, 0111 = 28 + 26 + 24 + 18 + 22 + 6 + 14 + 0 = 138 / 8 = 17.25 avg
# 1*** = 1000, 1001, 1010, 1011, 1100, 1101, 1110, 1111 = 20 + 12 + 10 + 2 + 8 + 4 + 6 + 30 = 92 / 8 = 11.5 avg

# 00** = 0000, 0001, 0010, 0011 = 28 + 26 + 24 + 18 = 96 / 4 = 24 avg
# 11** = 1100, 1101, 1110, 1111 = 8 + 4 + 6 + 30 = 48 / 4 = 12 avg

# *0*0 = 0000, 0010, 1000, 1010 = 28 + 24 + 20 + 10 = 82 / 4 = 20.5
# *1*1 = 0101, 0111, 1101, 1111 = 6 + 0 + 4 + 30 = 40 / 4 = 10 avg

## 3) FELADAT (könnyű) – GA egy generáció kézzel

*   Adj meg egy 6 elemű populációt 4 bites kromoszómákkal.
*   Számold ki a fitness értékeket, majd **roulette-wheel szelekcióval** válaszd ki 6 utód „szülőt”.
*   Nincs keresztezés, nincs mutáció – csak nézd meg, hogyan tolódik a populáció eloszlása.

Megjegyzés: Roulette-szelekcióra korábban adtam már példát. Azt lehet használni, mint inspirációt.



In [5]:
population = ['0110', '1110', '1010', '0111', '0010', '1101']
fitnesses = [f4[x] for x in population]
total_fitness = sum(fitnesses)
p = [f / total_fitness for f in fitnesses]
g2 = random.choices(population, weights=p, k=6)
g2

['1101', '0111', '0111', '1110', '0111', '1101']

## 4) FELADAT (közepes) – GA implementáció 4 bitre


*   Írj egy egyszerű GA-t 4 bitre (pl. populáció=10, generációk=20).
*   Keresztezés: egypontos, mutáció: bit-flip 1%.
*   Figyeld meg, hányszor találja meg a 0000-t 50 futásból (statisztika).



In [7]:
def ga(pop_size, max_generations, mutation_rate, crossover_rate, dna_format=8):

    population = create_starting_population(pop_size, dna_format)

    for generation in range(max_generations):

        fitness_scores = []
        for person in population:
            score = f4[bit_not(person)]

            fitness_scores.append(score)

        max_fit = max(fitness_scores)
        avg_fit = sum(fitness_scores) / len(fitness_scores)

        best_person_index = fitness_scores.index(max_fit)
        best_person = population[best_person_index]

        next_generation = []

        while len(next_generation) < pop_size:
            parent1 = pick_winner(population, fitness_scores, 3)
            parent2 = pick_winner(population, fitness_scores, 3)

            child1, child2 = crossover(parent1, parent2, crossover_rate)

            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            next_generation.append(child1)
            next_generation.append(child2)

        population = next_generation[:pop_size]

    return best_person

db = 0
for run in range(0, 50):
  i = ga(10, 20, 0.01, 0.8, 4)
  if i == '0000':
    db = db + 1
print(db, '/', 50)


29 / 50


## 5) FELADAT (közepes) – Sémaszintű vizsgálat (8 bit)


*   A f8 függvényhez válaszd a sémát: ****0000.
*   Számold ki az átlagfitneszt.
*   Majd cseréld az összes 0-t 1-re: ****1111, és számold ki az átlagot.
*   Értelmezd az eredményt: mit „lát” a GA az utolsó 4 bitről?




In [9]:
from itertools import product
DF2 = {
    "0000": 28, "0001": 26, "0010": 24, "0011": 18,
    "0100": 22, "0101":  6, "0110": 14, "0111":  0,
    "1000": 20, "1001": 12, "1010": 10, "1011":  2,
    "1100":  8, "1101":  4, "1110":  6, "1111": 30,
}

def bit_not(bits):
    return "".join("1" if b == "0" else "0" for b in bits)

f4 = {b: DF2[bit_not(b)] for b in DF2}

def create_bitstrings(bitek, include_star = False):
  if include_star:
    return ["".join(b) for b in product("01*", repeat=bitek)]
  return ["".join(b) for b in product("01", repeat=bitek)]

def flip_zeros_to_ones(schema):
  return ''.join(['1' if letter == '0' else letter for letter in schema])

def avg_fitness_of_schema(schema, bitek):
  good = []
  for string in create_bitstrings(bitek):
    matches = True
    for i in range(bitek):
      if schema[i] != '*' and schema[i] != string[i]:
        matches = False
        break
    if matches:
      good.append(string)
  if len(good) == 0:
    return 0
  return sum([f8(string) for string in good]) / len(good)

fit = avg_fitness_of_schema("****0000", 8)
flip = avg_fitness_of_schema("****1111", 8)
print('alap:', fit, "flip:", flip)

alap: 44.375 flip: 42.375


In [ ]:
# ga alapjan 0-soknak nagyobb a fitness erteke

## 6) FELADAT (közepes) – Mutáció hatása


*   Ugyanazzal a GA-val (4 vagy 8 bit) futtass két szimulációt:
*   (a) 𝑝𝑚𝑢𝑡 = 0.001
*   (b) 𝑝𝑚𝑢𝑡 = 0.05
*   Hasonlítsd össze, melyik esetben könnyebb a 0000 illetve 00000000 megtalálása.
*   Írd le 3–4 mondatban, hogy miért.




In [11]:
db = 0
for run in range(0, 50):
  i = ga(10, 20, 0.001, 0.8, 4)
  if i == '0000':
    db = db + 1
print(db,'/','50')

db = 0
for run in range(0, 50):
  i = ga(10, 20, 0.05, 0.8, 4)
  if i == '0000':
    db = db + 1
print(db,'/','50')

25 / 50
37 / 50


In [ ]:
# nagyobb mutacios rata neha atloki a megoldast a jo 0000 iranyba, ezert tobbszor megtalalja

## 7) FELADAT (haladó) – Mutáció hatása


*   Készíts egy módosított fitness-függvényt:
*   f4′​(x)=f4​(x)+zaj, ahol a zaj pl. −1,0,+1−1,0,+1 egyenletes eloszlásból.
*   Vizsgáld meg, hogy a zaj hatására javul-e vagy romlik-e a GA esélye a globális optimum elérésére.

Javaslat: futtass 100 szimulációt zajjal és zaj nélkül.



In [12]:
def ga(pop_size, max_generations, mutation_rate, crossover_rate, dna_format=8, add_noise = True):

    population = create_starting_population(pop_size, dna_format)

    for generation in range(max_generations):

        fitness_scores = []
        for person in population:
            if add_noise:
              score = f4[bit_not(person)] + random.random() * 2 - 1

            fitness_scores.append(score)

        max_fit = max(fitness_scores)
        avg_fit = sum(fitness_scores) / len(fitness_scores)

        best_person_index = fitness_scores.index(max_fit)
        best_person = population[best_person_index]

        next_generation = []

        while len(next_generation) < pop_size:
            parent1 = pick_winner(population, fitness_scores, 3)
            parent2 = pick_winner(population, fitness_scores, 3)

            child1, child2 = crossover(parent1, parent2, crossover_rate)

            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            next_generation.append(child1)
            next_generation.append(child2)

        population = next_generation[:pop_size]

    return best_person
db = 0
for run in range(0, 50):
  i = ga(10, 20, 0.05, 0.8, 4)
  if i == '0000':
    db = db + 1
print(db,'/','50')
db = 0
for run in range(0, 50):
  i = ga(10, 20, 0.05, 0.8, 4, True)
  if i == '0000':
    db = db + 1
print(db,'/','50')

33 / 50
34 / 50


In [ ]:
# zaj hasonlo hatassal van mint a mutacios rata, gyakrabban megtalalja az optimumot

## 8) FELADAT (haladó) – Más kódolás vizsgálata


*   Definiálj egy **grayscale-szerű leképezést**: 0000 → 0, 1111 → 15, és egy **unimodális függvényt** ezen az intervallumon (pl. konkáv parabolát).
*   Mutasd meg, hogy ugyanarra az optimumra egy másik kódolás már nem deceptív, a GA könnyen találja a megoldást.
*   Hasonlítsd össze futási eredményekkel.



In [15]:

def f8d(bits):
    return f4[bits[:4]] + f4[bits[4:]]

def f8u(bits):
    left_part = bits[:4]
    right_part = bits[4:]
    left_number = int(left_part, 2)
    right_number = int(right_part, 2)
    score = left_number * right_number
    return score

import random
def create_random_person(dna_length=8):
    dna = ""
    for i in range(dna_length):
        choice = random.choice(['0', '1'])
        dna += choice
    return dna

def create_starting_population(pop_size, dna_length=8):
    population = []
    for i in range(pop_size):
        new_person = create_random_person(dna_length)
        population.append(new_person)
    return population

def pick_winner(population, fitness_scores, k=3):
    contestant_indices = random.sample(range(len(population)), k)
    best_index = -1
    highest_score = -1

    for index in contestant_indices:
        score = fitness_scores[index]
        if score > highest_score:
            highest_score = score
            best_index = index

    return population[best_index]

def crossover(parent1, parent2, crossover_rate=0.7):
    if random.random() < crossover_rate:
        cut_point = random.randint(1, len(parent1) - 1)

        child1 = parent1[:cut_point] + parent2[cut_point:]
        child2 = parent2[:cut_point] + parent1[cut_point:]

        return child1, child2

    return parent1, parent2

def mutate(individual, mutation_rate=0.01):
    dna_chars = list(individual)

    for i in range(len(dna_chars)):
        if random.random() < mutation_rate:
            if dna_chars[i] == '0':
                dna_chars[i] = '1'
            else:
                dna_chars[i] = '0'

    return "".join(dna_chars)

def ga(pop_size, max_generations, mutation_rate, crossover_rate, dna_format=8, function = 'd'):

    population = create_starting_population(pop_size, dna_format)
    print("Gen   | Max Fit    | Best Person     | Avg Fit")
    print("-" * 50)

    for generation in range(max_generations):

        fitness_scores = []
        for person in population:
            if function == 'd':
              score = f8d(person)
            else:
              score = f8u(person)

            fitness_scores.append(score)

        max_fit = max(fitness_scores)
        avg_fit = sum(fitness_scores) / len(fitness_scores)

        best_person_index = fitness_scores.index(max_fit)
        best_person = population[best_person_index]

        print(f"{generation}     | {max_fit:.2f}       | {best_person} | {avg_fit:.2f}")

        next_generation = []

        while len(next_generation) < pop_size:
            parent1 = pick_winner(population, fitness_scores, 3)
            parent2 = pick_winner(population, fitness_scores, 3)

            child1, child2 = crossover(parent1, parent2, crossover_rate)

            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            next_generation.append(child1)
            next_generation.append(child2)

        population = next_generation[:pop_size]

    print("\nMax generations reached.")

    return best_person

ga(20, 50, 0.05, 0.8, 8, 'd')
ga(20, 50, 0.05, 0.8, 8, 'u')

Gen   | Max Fit    | Best Person     | Avg Fit
--------------------------------------------------
0     | 42.00       | 11011100 | 21.40
1     | 52.00       | 11011111 | 32.20
2     | 52.00       | 11111101 | 38.60
3     | 52.00       | 11111101 | 39.30
4     | 56.00       | 11111111 | 48.80
5     | 58.00       | 11110000 | 48.90
6     | 58.00       | 11110000 | 47.40
7     | 58.00       | 11110000 | 47.40
8     | 58.00       | 11110000 | 48.00
9     | 58.00       | 11110000 | 50.90
10     | 58.00       | 11110000 | 48.90
11     | 58.00       | 11110000 | 48.80
12     | 58.00       | 11110000 | 49.80
13     | 58.00       | 11110000 | 49.80
14     | 58.00       | 11110000 | 49.50
15     | 58.00       | 11110000 | 49.70
16     | 58.00       | 11110000 | 51.30
17     | 58.00       | 11110000 | 51.60
18     | 58.00       | 11110000 | 51.70
19     | 58.00       | 11110000 | 50.20
20     | 58.00       | 11110000 | 50.60
21     | 58.00       | 11110000 | 49.40
22     | 58.00       | 11110000 

'11111111'

## 9) FELADAT (haladó) – 8 bites blokkok kombinálása


*   A fenti 𝑓8-at módosítsd úgy, hogy az első blokkon f4, a második blokkon pedig egy egyszerű **hill-climbing-barát függvény** fusson (pl. az 1-ek száma). Mit is takar pontosan a **hill-climbing-barát függvény**? Kérem kifejteni.
*   Figyeld meg, hogy a GA az egyik blokkban „elcsúszik” (deceptív), a másikban jól dolgozik.
*  Írd le, mit tanulhatunk ebből a problémakódolásról és a GA korlátairól.



In [16]:
def f8d(bits: str) -> int:
    assert len(bits) == 8
    return f4[bits[:4]] + sum(int(bit) for bit in bits[4:])

ga(20, 50, 0.05, 0.8, 8, 'd')

Gen   | Max Fit    | Best Person     | Avg Fit
--------------------------------------------------
0     | 30.00       | 11111001 | 15.45
1     | 31.00       | 11111011 | 24.45
2     | 31.00       | 11111011 | 29.05
3     | 31.00       | 11111011 | 28.85
4     | 32.00       | 11111111 | 29.40
5     | 31.00       | 11111011 | 29.80
6     | 32.00       | 11111111 | 29.10
7     | 32.00       | 11111111 | 29.15
8     | 32.00       | 11111111 | 30.40
9     | 32.00       | 11111111 | 29.90
10     | 32.00       | 11111111 | 31.40
11     | 32.00       | 11111111 | 30.80
12     | 32.00       | 11111111 | 29.90
13     | 32.00       | 11111111 | 29.80
14     | 32.00       | 11111111 | 31.10
15     | 32.00       | 11111111 | 29.55
16     | 32.00       | 11111111 | 30.15
17     | 32.00       | 11111111 | 30.70
18     | 32.00       | 11111111 | 29.60
19     | 32.00       | 11111111 | 31.05
20     | 32.00       | 11111111 | 30.40
21     | 32.00       | 11111111 | 30.25
22     | 32.00       | 11111111 

'11111111'

In [ ]:
# fennall a veszelye, hogy az algoritmus beleragad egy "lokalis optimumba" (csapdaba),
# es nem talalja meg a legjobb megoldast.

# a deceptiv fuggveny egy olyan megoldas, ami elkeruli ezeket a csapdakat,
# es biztosan a "globalis optimum" (a tenyleges legjobb) fele halad.